In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import math
%matplotlib inline
import seaborn as sns
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA

In [2]:
df_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
df_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')
IDtest = df_test.custid.unique()

train_id = df_train['custid']
test_id = df_test['custid']

df_all = pd.concat([df_train, df_test])

df_all.head()

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0


In [3]:
features = pd.DataFrame({'custid': df_all.custid.unique()})

### PCA

In [4]:
# 차원축소 매소드 
from sklearn.decomposition import PCA

def dummy_to_pca(tr, column_name:str, features) :
    max_seq = 300
    max_d = 15
    col_count = tr.groupby(column_name)[column_name].count()
    if len(col_count) > max_seq:
        tops = col_count.sort_values(ascending=False)[0:max_seq].index
        f =tr.loc[tr[column_name].isin(tops)][['custid', column_name]]
    else:
        tops = col_count.index
        f =tr[['custid', column_name]]
    f = pd.get_dummies(f, columns=[column_name])  # This method performs One-hot-encoding
    f = f.groupby('custid').mean()
    if len(tops) < max_d:
        max_d = len(tops)
    pca = PCA(n_components=max_d)
    pca.fit(f)
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
    #print(cumsum)
    num_d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 차원의 수
    if num_d == 1:
        num_d = max_d
    pca = PCA(n_components=num_d)    
    result = pca.fit_transform(f)
    result = pd.DataFrame(result)
    result.columns = [column_name + '_' + str(column) for column in result.columns]
    result.index = f.index
    return result.reset_index()

#f = dummy_to_pca(tr, 'team_nm', f)
#f

### Make features

In [5]:
df_all['date'] = df_all['sales_month']*100+df_all['sales_day']

In [6]:
df_all['refund_tot'] = np.where(df_all['tot_amt']<0, (-1)*df_all['tot_amt'],0)



In [7]:
f = df_all.groupby('custid').agg({
    'tot_amt': [('총구매액', 'sum'),('구매건수', 'size'),('평균구매가격', 'mean'),('최대구매액', 'max')],
    'dis_amt': [('dis_sum', 'sum'),('dis_mean', 'mean')],
    'net_amt': [('net_sum', 'sum'),('net_mean', 'mean')],
    'inst_mon': [('평균할부개월수', 'mean'), ('최대할부개월수', 'max')],
    'brd_nm': [('구매상품다양성비', lambda x: x.nunique()/x.count())],
    'str_nm': [('지점별다양성비', lambda x: x.nunique()/x.count())],
    'corner_nm': [('코너별다양성비', lambda x: x.nunique()/x.count())],
    'pc_nm': [('상품군별다양성비', lambda x: x.nunique()/x.count())],
    'part_nm': [('상품파트별다양성비', lambda x: x.nunique()/x.count())],
    'team_nm': [('상품팀별다양성비', lambda x: x.nunique()/x.count())],
    'buyer_nm': [('바이어별다양성비', lambda x: x.nunique()/x.count())],
    'import_flg': [('수입상품_구매비율', "mean"), ('수입상품_최대', 'max'), ('수입상품_최소', 'min')],
    'inst_fee' : [('무이자_구매비율', "mean"), ('무이자_최대', 'max'), ('무이자_최소', 'min')],
    'sales_month': [
        ('봄-구매비율', lambda x: np.mean( x.isin([15,16,5]))),
        ('여름-구매비율', lambda x: np.mean( x.isin([6,7,8]))),
        ('가을-구매비율', lambda x: np.mean( x.isin([9,10,11]))),
        ('겨울-구매비율', lambda x: np.mean( x.isin([13,14,12]))),
    ],
    'sales_dayofweek': [
        ('주말방문비율', lambda x: np.mean(x.isin(['토','일']))),
    ],
    'date': [
        ('내점일수',lambda x: x.nunique()),
        ('내점비율',lambda x: x.nunique()/x.count()),
    ],
    'sales_time': [('밤구입비율', lambda x: np.count_nonzero([(x>1800)|(x<900)])/ x.count())],
    }).reset_index()
features = features.merge(f, on='custid'); features




,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,"(inst_fee, 무이자_최대)","(inst_fee, 무이자_최소)","(sales_month, 봄-구매비율)","(sales_month, 여름-구매비율)","(sales_month, 가을-구매비율)","(sales_month, 겨울-구매비율)","(sales_dayofweek, 주말방문비율)","(date, 내점일수)","(date, 내점비율)","(sales_time, 밤구입비율)"
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,0.090909,0.363636,0.272727,0.272727,0.636364,7,0.636364,0.818182
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,1,0,0.000000,0.272727,0.454545,0.272727,0.363636,7,0.636364,0.363636
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,1,0,0.300000,0.400000,0.100000,0.200000,0.366667,13,0.433333,0.400000
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,1,0,0.000000,1.000000,0.000000,0.000000,0.250000,2,0.500000,0.000000
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0,0,0.187500,0.406250,0.375000,0.031250,0.156250,21,0.656250,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0.000000,0.000000,0.750000,0.250000,0.000000,2,0.500000,0.000000
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,1,1,0.000000,0.000000,0.000000,1.000000,0.000000,1,1.000000,0.000000
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0.500000,0.500000,0.000000,0.000000,0.500000,2,1.000000,0.500000
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0.000000,0.000000,0.000000,1.000000,0.000000,1,0.250000,0.250000


### PCA

In [8]:
df_all['month'] = df_all['sales_month'].astype(str)
df_all['week'] = df_all['sales_dayofweek'].astype(str)
df_all['time'] = (df_all['sales_time']/100).astype(int).astype(str)
df_all['inst_mon1'] = df_all['inst_mon'].astype(str)
df_all['inst_fee1'] = df_all['inst_fee'].astype(str)

In [9]:
f = dummy_to_pca(df_all, 'brd_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'corner_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'pc_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'part_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'buyer_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'team_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'goodcd', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'str_nm', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'month', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'week', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'time', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'inst_mon1', features)
features = features.merge(f, how='left'); features
f = dummy_to_pca(df_all, 'inst_fee1', features)
features = features.merge(f, how='left'); features
features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,time_6,time_7,time_8,inst_mon1_0,inst_mon1_1,inst_mon1_2,inst_mon1_3,inst_mon1_4,inst_fee1_0,inst_fee1_1
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,-0.058412,-0.111035,-0.010832,0.742014,-0.059197,-0.000834,-0.003930,0.000845,-0.040518,4.360643e-15
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,-0.008219,0.100021,0.163474,0.045503,0.230536,-0.051186,-0.009049,0.028437,0.345177,1.026319e-17
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,-0.056750,0.017088,-0.021126,0.425770,0.020682,0.051679,0.078422,-0.030475,0.100903,1.242460e-17
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,-0.340758,-0.050109,-0.006280,0.004852,0.463112,0.421214,0.111618,0.107315,0.666589,1.522361e-18
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,-0.057418,0.178548,-0.044517,0.075575,-0.033983,-0.003424,-0.000393,0.000709,-0.040518,-8.670979e-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,-0.238935,-0.463125,-0.007786,0.345318,0.123460,0.211202,-0.140741,-0.025506,-0.040518,-8.670979e-18
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.107354,-0.007981,-0.005764,0.208206,1.108218,-0.405498,-0.116907,-0.097986,1.373696,-4.063958e-17
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,-0.007050,-0.004650,-0.004289,-0.542687,-0.010592,-0.005827,0.002889,0.000582,-0.040518,-8.670979e-18
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,-0.004750,-0.006911,-0.004394,-0.542687,-0.010592,-0.005827,0.002889,0.000582,-0.040518,-8.670979e-18


In [10]:
list(features.columns)[9]

('inst_mon', '평균할부개월수')

In [11]:
list(features.columns)[2]

('tot_amt', '구매건수')

In [12]:
features['내점당구매액'] = features[list(features.columns)[1]]/features[list(features.columns)[29]]
features['내점당평균구매액'] = features[list(features.columns)[3]]/features[list(features.columns)[29]]
features['내점당구매건수'] = features[list(features.columns)[2]]/features[list(features.columns)[29]];features



,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,inst_mon1_0,inst_mon1_1,inst_mon1_2,inst_mon1_3,inst_mon1_4,inst_fee1_0,inst_fee1_1,내점당구매액,내점당평균구매액,내점당구매건수
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.742014,-0.059197,-0.000834,-0.003930,0.000845,-0.040518,4.360643e-15,248857.142857,22623.376623,1.571429
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.045503,0.230536,-0.051186,-0.009049,0.028437,0.345177,1.026319e-17,535835.714286,48712.337662,1.571429
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.425770,0.020682,0.051679,0.078422,-0.030475,0.100903,1.242460e-17,176961.538462,5898.717949,2.307692
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.004852,0.463112,0.421214,0.111618,0.107315,0.666589,1.522361e-18,522500.000000,130625.000000,2.000000
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.075575,-0.033983,-0.003424,-0.000393,0.000709,-0.040518,-8.670979e-18,240655.190476,7520.474702,1.523810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.345318,0.123460,0.211202,-0.140741,-0.025506,-0.040518,-8.670979e-18,316000.000000,79000.000000,2.000000
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.208206,1.108218,-0.405498,-0.116907,-0.097986,1.373696,-4.063958e-17,213000.000000,213000.000000,1.000000
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,-0.542687,-0.010592,-0.005827,0.002889,0.000582,-0.040518,-8.670979e-18,110500.000000,55250.000000,1.000000
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,-0.542687,-0.010592,-0.005827,0.002889,0.000582,-0.040518,-8.670979e-18,143029.000000,35757.250000,4.000000


In [13]:
f = df_all.query("tot_amt < 0").groupby('custid')['tot_amt'].agg([
    ('환불금액', np.sum)
]).reset_index()
f['환불금액'] = f['환불금액']*(-1)
features = features.merge(f, how='left')
features['환불금액'] = features['환불금액'].fillna(0)

f = df_all.query("tot_amt < 0").groupby('custid')['tot_amt'].agg([
    ('환불건수', np.size)
]).reset_index()
features = features.merge(f, how='left')
features['환불건수'] = features['환불건수'].fillna(0)
features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,inst_mon1_2,inst_mon1_3,inst_mon1_4,inst_fee1_0,inst_fee1_1,내점당구매액,내점당평균구매액,내점당구매건수,환불금액,환불건수
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,-0.000834,-0.003930,0.000845,-0.040518,4.360643e-15,248857.142857,22623.376623,1.571429,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,-0.051186,-0.009049,0.028437,0.345177,1.026319e-17,535835.714286,48712.337662,1.571429,1850500.0,2.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.051679,0.078422,-0.030475,0.100903,1.242460e-17,176961.538462,5898.717949,2.307692,695600.0,2.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.421214,0.111618,0.107315,0.666589,1.522361e-18,522500.000000,130625.000000,2.000000,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,-0.003424,-0.000393,0.000709,-0.040518,-8.670979e-18,240655.190476,7520.474702,1.523810,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.211202,-0.140741,-0.025506,-0.040518,-8.670979e-18,316000.000000,79000.000000,2.000000,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,-0.405498,-0.116907,-0.097986,1.373696,-4.063958e-17,213000.000000,213000.000000,1.000000,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,-0.005827,0.002889,0.000582,-0.040518,-8.670979e-18,110500.000000,55250.000000,1.000000,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,-0.005827,0.002889,0.000582,-0.040518,-8.670979e-18,143029.000000,35757.250000,4.000000,0.0,0.0


In [14]:
features['환불비율'] = features['환불건수']/features[list(features.columns)[2]]
features['내점당환불액'] = features['환불금액']/features[list(features.columns)[29]]
features['내점당환불건수'] = features['환불건수']/features[list(features.columns)[29]]
features['구매건수대비평균환불액'] = features['환불금액']/features[list(features.columns)[2]]
features['한달평균납부액'] = features[list(features.columns)[1]]/features[list(features.columns)[9]]
features['한달결제취소액'] = features['환불금액']/features[list(features.columns)[9]];features



,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,내점당평균구매액,내점당구매건수,환불금액,환불건수,환불비율,내점당환불액,내점당환불건수,구매건수대비평균환불액,한달평균납부액,한달결제취소액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,22623.376623,1.571429,0.0,0.0,0.000000,0.000000,0.000000,0.000000,6.181290e+05,0.000000
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,48712.337662,1.571429,1850500.0,2.0,0.181818,264357.142857,0.285714,168227.272727,1.085772e+06,535671.052632
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,5898.717949,2.307692,695600.0,2.0,0.066667,53507.692308,0.153846,23186.666667,8.626875e+05,260850.000000
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,130625.000000,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.322222e+05,0.000000
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,7520.474702,1.523810,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.695338e+06,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,79000.000000,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.808889e+05,0.000000
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,213000.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.550000e+04,0.000000
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,55250.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.210000e+05,0.000000
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,35757.250000,4.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.430290e+05,0.000000


In [15]:
f = df_all.query("tot_amt > 0").groupby('custid')['tot_amt'].agg([
    ('최소구매액', np.min)
]).reset_index()

features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,내점당구매건수,환불금액,환불건수,환불비율,내점당환불액,내점당환불건수,구매건수대비평균환불액,한달평균납부액,한달결제취소액,최소구매액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,1.571429,0.0,0.0,0.000000,0.000000,0.000000,0.000000,6.181290e+05,0.000000,39000
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,1.571429,1850500.0,2.0,0.181818,264357.142857,0.285714,168227.272727,1.085772e+06,535671.052632,146000
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,2.307692,695600.0,2.0,0.066667,53507.692308,0.153846,23186.666667,8.626875e+05,260850.000000,4200
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.322222e+05,0.000000,49000
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,1.523810,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.695338e+06,0.000000,20335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.808889e+05,0.000000,52000
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.550000e+04,0.000000,213000
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.210000e+05,0.000000,53000
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,4.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.430290e+05,0.000000,20000


무이자

In [16]:
f = pd.crosstab(df_all.custid,df_all.inst_fee)
f.columns = ['무이자','이자']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,환불건수,환불비율,내점당환불액,내점당환불건수,구매건수대비평균환불액,한달평균납부액,한달결제취소액,최소구매액,무이자,이자
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.000000,0.000000,0.000000,0.000000,6.181290e+05,0.000000,39000,11,0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,2.0,0.181818,264357.142857,0.285714,168227.272727,1.085772e+06,535671.052632,146000,8,3
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,2.0,0.066667,53507.692308,0.153846,23186.666667,8.626875e+05,260850.000000,4200,27,3
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.000000,0.000000,0.000000,0.000000,2.322222e+05,0.000000,49000,2,2
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.0,0.000000,0.000000,0.000000,0.000000,2.695338e+06,0.000000,20335,32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.000000,0.000000,0.000000,0.000000,2.808889e+05,0.000000,52000,4,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,3.550000e+04,0.000000,213000,0,1
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,2.210000e+05,0.000000,53000,2,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,1.430290e+05,0.000000,20000,4,0


In [17]:
f = pd.crosstab(df_all.custid, df_all.inst_fee, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['무이자합계금액','이자합계금액']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,내점당환불액,내점당환불건수,구매건수대비평균환불액,한달평균납부액,한달결제취소액,최소구매액,무이자,이자,무이자합계금액,이자합계금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.000000,0.000000,0.000000,6.181290e+05,0.000000,39000,11,0,1742000.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,264357.142857,0.285714,168227.272727,1.085772e+06,535671.052632,146000,8,3,2550850.0,1200000.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,53507.692308,0.153846,23186.666667,8.626875e+05,260850.000000,4200,27,3,1761500.0,539000.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.000000,0.000000,0.000000,2.322222e+05,0.000000,49000,2,2,109000.0,936000.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.000000,0.000000,0.000000,2.695338e+06,0.000000,20335,32,0,5053759.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.000000,0.000000,0.000000,2.808889e+05,0.000000,52000,4,0,632000.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.000000,0.000000,0.000000,3.550000e+04,0.000000,213000,0,1,0.0,213000.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.000000,0.000000,0.000000,2.210000e+05,0.000000,53000,2,0,221000.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.000000,0.000000,0.000000,1.430290e+05,0.000000,20000,4,0,143029.0,0.0


In [18]:
f = pd.crosstab(df_all.custid, df_all.inst_fee, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['무이자합계할인금액','이자합계할인금액']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,구매건수대비평균환불액,한달평균납부액,한달결제취소액,최소구매액,무이자,이자,무이자합계금액,이자합계금액,무이자합계할인금액,이자합계할인금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.000000,6.181290e+05,0.000000,39000,11,0,1742000.0,0.0,174200.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,168227.272727,1.085772e+06,535671.052632,146000,8,3,2550850.0,1200000.0,255090.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,23186.666667,8.626875e+05,260850.000000,4200,27,3,1761500.0,539000.0,64710.0,26950.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.000000,2.322222e+05,0.000000,49000,2,2,109000.0,936000.0,3000.0,18800.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.000000,2.695338e+06,0.000000,20335,32,0,5053759.0,0.0,361260.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.000000,2.808889e+05,0.000000,52000,4,0,632000.0,0.0,31600.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.000000,3.550000e+04,0.000000,213000,0,1,0.0,213000.0,0.0,10650.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.000000,2.210000e+05,0.000000,53000,2,0,221000.0,0.0,11050.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.000000,1.430290e+05,0.000000,20000,4,0,143029.0,0.0,3500.0,0.0


수입상품분류

In [19]:
f = pd.crosstab(df_all.custid,df_all.import_flg)
f.columns = ['국내상품','수입상품']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,한달결제취소액,최소구매액,무이자,이자,무이자합계금액,이자합계금액,무이자합계할인금액,이자합계할인금액,국내상품,수입상품
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.000000,39000,11,0,1742000.0,0.0,174200.0,0.0,4,7
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,535671.052632,146000,8,3,2550850.0,1200000.0,255090.0,0.0,10,1
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,260850.000000,4200,27,3,1761500.0,539000.0,64710.0,26950.0,30,0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.000000,49000,2,2,109000.0,936000.0,3000.0,18800.0,3,1
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.000000,20335,32,0,5053759.0,0.0,361260.0,0.0,26,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.000000,52000,4,0,632000.0,0.0,31600.0,0.0,3,1
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.000000,213000,0,1,0.0,213000.0,0.0,10650.0,0,1
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.000000,53000,2,0,221000.0,0.0,11050.0,0.0,1,1
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.000000,20000,4,0,143029.0,0.0,3500.0,0.0,4,0


In [20]:
f = pd.crosstab(df_all.custid,df_all.import_flg, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['국내상품합계금액','수입상품합계금액']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,무이자,이자,무이자합계금액,이자합계금액,무이자합계할인금액,이자합계할인금액,국내상품,수입상품,국내상품합계금액,수입상품합계금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,11,0,1742000.0,0.0,174200.0,0.0,4,7,625000.0,1117000.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,8,3,2550850.0,1200000.0,255090.0,0.0,10,1,3604850.0,146000.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,27,3,1761500.0,539000.0,64710.0,26950.0,30,0,2300500.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,2,2,109000.0,936000.0,3000.0,18800.0,3,1,985000.0,60000.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,32,0,5053759.0,0.0,361260.0,0.0,26,6,3564559.0,1489200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,4,0,632000.0,0.0,31600.0,0.0,3,1,534000.0,98000.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,1,0.0,213000.0,0.0,10650.0,0,1,0.0,213000.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,2,0,221000.0,0.0,11050.0,0.0,1,1,168000.0,53000.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,4,0,143029.0,0.0,3500.0,0.0,4,0,143029.0,0.0


In [21]:
f = pd.crosstab(df_all.custid,df_all.import_flg, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['국내상품합계할인금액','수입상품합계할인금액']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,무이자합계금액,이자합계금액,무이자합계할인금액,이자합계할인금액,국내상품,수입상품,국내상품합계금액,수입상품합계금액,국내상품합계할인금액,수입상품합계할인금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,1742000.0,0.0,174200.0,0.0,4,7,625000.0,1117000.0,62500.0,111700.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,2550850.0,1200000.0,255090.0,0.0,10,1,3604850.0,146000.0,240490.0,14600.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,1761500.0,539000.0,64710.0,26950.0,30,0,2300500.0,0.0,91660.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,109000.0,936000.0,3000.0,18800.0,3,1,985000.0,60000.0,18800.0,3000.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,5053759.0,0.0,361260.0,0.0,26,6,3564559.0,1489200.0,233590.0,127670.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,632000.0,0.0,31600.0,0.0,3,1,534000.0,98000.0,26700.0,4900.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,213000.0,0.0,10650.0,0,1,0.0,213000.0,0.0,10650.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,221000.0,0.0,11050.0,0.0,1,1,168000.0,53000.0,8400.0,2650.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,143029.0,0.0,3500.0,0.0,4,0,143029.0,0.0,3500.0,0.0


할부개월분류

In [22]:
f = pd.crosstab(df_all.custid,df_all.inst_mon).fillna(0)
f.columns = ['할부1개월','할부2개월','할부3개월','할부4개월','할부5개월','할부6개월','할부7개월','할부8개월','할부9개월','할부10개월','할부11개월','할부12개월']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,할부3개월,할부4개월,할부5개월,할부6개월,할부7개월,할부8개월,할부9개월,할부10개월,할부11개월,할부12개월
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,10,0,0,0,0,0,0,0,0,0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,3,0,0,2,0,0,0,0,0,1
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,19,0,3,0,0,0,0,0,0,0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,1,0,0,0,0,1,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,14,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,2,0,0,0,0,0,0,0,0,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,1,0,0,0,0,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,0,0,0,0,0,0


In [23]:
f = pd.crosstab(df_all.custid, df_all.inst_mon, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['할부1개월합계금액','할부2개월합계금액','할부3개월합계금액','할부4개월합계금액','할부5개월합계금액','할부6개월합계금액','할부7개월합계금액','할부8개월합계금액','할부9개월합계금액','할부10개월합계금액','할부11개월합계금액','할부12개월합계금액']
features = features.merge(f, on = 'custid'); features


,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,할부3개월합계금액,할부4개월합계금액,할부5개월합계금액,할부6개월합계금액,할부7개월합계금액,할부8개월합계금액,할부9개월합계금액,할부10개월합계금액,할부11개월합계금액,할부12개월합계금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,1703000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,888350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1200000.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,1572700.0,0.0,539000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,376000.0,0.0,0.0,0.0,0.0,560000.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,3608000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,521000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,213000.0,0.0,0.0,0.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
f = pd.crosstab(df_all.custid, df_all.inst_mon, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['할부1개월합계할인금액','할부2개월합계할인금액','할부3개월합계할인금액','할부4개월합계할인금액','할부5개월합계할인금액','할부6개월합계할인금액','할부7개월합계할인금액','할부8개월합계할인금액','할부9개월합계할인금액','할부10개월합계할인금액','할부11개월합계할인금액','할부12개월합계할인금액']
features = features.merge(f, on = 'custid'); features


,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,할부3개월합계할인금액,할부4개월합계할인금액,할부5개월합계할인금액,할부6개월합계할인금액,할부7개월합계할인금액,할부8개월합계할인금액,할부9개월합계할인금액,할부10개월합계할인금액,할부11개월합계할인금액,할부12개월합계할인금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,170300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,88840.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,57910.0,0.0,26950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,18800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,259550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,26050.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,10650.0,0.0,0.0,0.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


월별방문횟수

In [25]:
f = pd.crosstab(df_all.custid, df_all.sales_month).fillna(0)
f.columns = ['5월','6월','7월','8월','9월','10월','11월','12월','1월','2월','3월','4월']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,7월,8월,9월,10월,11월,12월,1월,2월,3월,4월
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,2,3,0,0,3,0,0,0,1
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,3,2,2,1,1,2,0,0,0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,7,4,3,0,0,4,2,0,3,4
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,4,0,0,0,0,0,0,0,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,6,4,5,5,2,1,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,0,3,0,1,0,0,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,0,0,0,0,1,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,1,0,0,0,0,0,0,0,0,1
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,0,0,4,0,0,0


In [26]:
f = pd.crosstab(df_all.custid, df_all.sales_month, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['5월합계금액','6월합계금액','7월합계금액','8월합계금액','9월합계금액','10월합계금액','11월합계금액','12월합계금액','1월합계금액','2월합계금액','3월합계금액','4월합계금액']
features = features.merge(f, on = 'custid'); features


,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,7월합계금액,8월합계금액,9월합계금액,10월합계금액,11월합계금액,12월합계금액,1월합계금액,2월합계금액,3월합계금액,4월합계금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,630000.0,331000.0,0.0,0.0,358000.0,0.0,0.0,0.0,294000.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,404500.0,1557000.0,357350.0,232000.0,1416000.0,-216000.0,0.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,337800.0,146200.0,134800.0,0.0,0.0,462500.0,143000.0,0.0,1295000.0,-394000.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,1045000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,1009380.0,672249.0,953335.0,602400.0,100812.0,58083.0,0.0,0.0,804000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0.0,573000.0,0.0,59000.0,0.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213000.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,53000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168000.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,143029.0,0.0,0.0,0.0


In [27]:
f = pd.crosstab(df_all.custid, df_all.sales_month, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['5월합계할인금액','6월합계할인금액','7월합계할인금액','8월합계할인금액','9월합계할인금액','10월합계할인금액','11월합계할인금액','12월합계할인금액','1월합계할인금액','2월합계할인금액','3월합계할인금액','4월합계할인금액']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,7월합계할인금액,8월합계할인금액,9월합계할인금액,10월합계할인금액,11월합계할인금액,12월합계할인금액,1월합계할인금액,2월합계할인금액,3월합계할인금액,4월합계할인금액
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,63000.0,33100.0,0.0,0.0,35800.0,0.0,0.0,0.0,29400.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,40450.0,155700.0,35740.0,23200.0,0.0,0.0,0.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,13550.0,5220.0,2950.0,0.0,0.0,20440.0,4450.0,0.0,64750.0,-19700.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,21800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,72700.0,29370.0,93300.0,55840.0,4300.0,0.0,0.0,0.0,63800.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0.0,28650.0,0.0,2950.0,0.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10650.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,2650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8400.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3500.0,0.0,0.0,0.0


요일별분류

In [28]:
f = pd.crosstab(df_all.custid, df_all.sales_dayofweek).fillna(0)
f.columns = ['금','목','수','월','일','토','화']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,2월합계할인금액,3월합계할인금액,4월합계할인금액,금,목,수,월,일,토,화
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,29400.0,2,0,1,1,5,2,0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,0.0,0.0,2,0,5,0,4,0,0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.0,64750.0,-19700.0,6,7,0,6,6,5,0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,0.0,0,3,0,0,1,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.0,63800.0,0.0,9,14,1,2,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0,0,0,0,0,0,4
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,10650.0,0.0,0.0,0,0,1,0,0,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,8400.0,0,1,0,0,1,0,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0,0,4,0,0,0,0


In [29]:
f = pd.crosstab(df_all.custid, df_all.sales_dayofweek, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['금요일금액합계','목요일금액합계','수요일금액합계','월요일금액합계','일요일금액합계','토요일금액합계','화요일금액합계']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,일,토,화,금요일금액합계,목요일금액합계,수요일금액합계,월요일금액합계,일요일금액합계,토요일금액합계,화요일금액합계
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,5,2,0,262000.0,0.0,96000.0,294000.0,460000.0,630000.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,4,0,0,2528000.0,0.0,586350.0,0.0,636500.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,6,5,0,898800.0,335200.0,0.0,-131500.0,800000.0,398000.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,1,0,0,0.0,485000.0,0.0,0.0,560000.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,3,2,1,1975380.0,2019149.0,57812.0,128000.0,771000.0,78418.0,24000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,632000.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,0.0,0.0,213000.0,0.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,1,0,0,0.0,168000.0,0.0,0.0,53000.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0.0,0.0,143029.0,0.0,0.0,0.0,0.0


In [30]:
f = pd.crosstab(df_all.custid, df_all.sales_dayofweek, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['금요일할인금액합계','목요일할인금액합계','수요일할인금액합계','월요일할인금액합계','일요일할인금액합계','토요일할인금액합계','화요일할인금액합계']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,일요일금액합계,토요일금액합계,화요일금액합계,금요일할인금액합계,목요일할인금액합계,수요일할인금액합계,월요일할인금액합계,일요일할인금액합계,토요일할인금액합계,화요일할인금액합계
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,460000.0,630000.0,0.0,26200.0,0.0,9600.0,29400.0,46000.0,63000.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,636500.0,0.0,0.0,111200.0,0.0,80240.0,0.0,63650.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,800000.0,398000.0,0.0,38450.0,13550.0,0.0,-10610.0,38920.0,11350.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,560000.0,0.0,0.0,0.0,21800.0,0.0,0.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,771000.0,78418.0,24000.0,169300.0,143060.0,0.0,12800.0,33700.0,0.0,2400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,632000.0,0.0,0.0,0.0,0.0,0.0,0.0,31600.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,0.0,0.0,10650.0,0.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,53000.0,0.0,0.0,0.0,8400.0,0.0,0.0,2650.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,0.0,3500.0,0.0,0.0,0.0,0.0


In [31]:
df_all['time'] = df_all['sales_time']//100
df_all['time'].unique()
df_all['time'] = np.where(df_all['time']<10, 10, df_all['time'])

In [32]:
f = pd.crosstab(df_all.custid, df_all.time).fillna(0)
f.columns = ['10시','11시','시12시','13시','14시','15시','16시','17시','18시','19시','20사','21시','22시']
features = features.merge(f, on = 'custid'); features


,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,13시,14시,15시,16시,17시,18시,19시,20사,21시,22시
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,0,0,0,4,5,0,0,0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,1,1,0,1,1,1,3,0,0,0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,2,3,0,1,5,11,2,0,0,0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,4,0,0,0,0,0,0,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,2,0,1,3,5,6,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,0,0,1,0,0,0,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,1,0,0,0,0,0,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0,0,1,1,0,0,0,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,1,2,1,0,0,0,0


In [33]:
f = pd.crosstab(df_all.custid, df_all.time, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['10시금액합계','11시금액합계','12시금액합계','13시금액합계','14시금액합계','15시금액합계','16시금액합계','17시금액합계','18시금액합계','19시금액합계','20사금액합계','21시금액합계','22시금액합계']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,13시금액합계,14시금액합계,15시금액합계,16시금액합계,17시금액합계,18시금액합계,19시금액합계,20사금액합계,21시금액합계,22시금액합계
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,0.0,0.0,0.0,843000.0,770000.0,0.0,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,445000.0,1112000.0,0.0,211350.0,146000.0,434500.0,1386000.0,0.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,166000.0,917500.0,0.0,-621000.0,274000.0,729000.0,566000.0,0.0,0.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,1045000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,573000.0,0.0,93000.0,283000.0,599980.0,927335.0,991300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0.0,0.0,59000.0,0.0,0.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,213000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,0.0,53000.0,168000.0,0.0,0.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,21000.0,52029.0,70000.0,0.0,0.0,0.0,0.0


In [34]:
f = pd.crosstab(df_all.custid, df_all.time, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['10시할인금액합계','11시할인금액합계','12시할인금액합계','13시할인금액합계','14시할인금액합계','15시할인금액합계','16시할인금액합계','17시할인금액합계','18시할인금액합계','19시할인금액합계','20시할인금액합계','21시할인금액합계','22시할인금액합계']
features = features.merge(f, on = 'custid'); features


,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,13시할인금액합계,14시할인금액합계,15시할인금액합계,16시할인금액합계,17시할인금액합계,18시할인금액합계,19시할인금액합계,20시할인금액합계,21시할인금액합계,22시할인금액합계
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,0.0,0.0,0.0,84300.0,77000.0,0.0,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,44500.0,111200.0,0.0,21140.0,14600.0,43450.0,-3000.0,0.0,0.0,0.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,8300.0,43190.0,0.0,-31050.0,12150.0,22450.0,26950.0,0.0,0.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,21800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,57300.0,0.0,9300.0,23900.0,55840.0,90700.0,15050.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0.0,0.0,2950.0,0.0,0.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,10650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,0.0,2650.0,8400.0,0.0,0.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,0.0,3500.0,0.0,0.0,0.0,0.0


지점별방문횟수

In [35]:
f = pd.crosstab(df_all.custid, df_all.str_nm).fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,17시할인금액합계,18시할인금액합계,19시할인금액합계,20시할인금액합계,21시할인금액합계,22시할인금액합계,무역점,본점,신촌점,천호점
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,84300.0,77000.0,0.0,0.0,0.0,5,6,0,0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,14600.0,43450.0,-3000.0,0.0,0.0,0.0,8,0,0,3
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,12150.0,22450.0,26950.0,0.0,0.0,0.0,4,0,0,26
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,55840.0,90700.0,15050.0,0.0,0.0,0.0,3,24,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,2950.0,0.0,0.0,0.0,0.0,0,4,0,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,2650.0,8400.0,0.0,0.0,0.0,0.0,0,2,0,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,3500.0,0.0,0.0,0.0,0.0,0,0,4,0


In [36]:
f = pd.crosstab(df_all.custid, df_all.str_nm, values = df_all.tot_amt, aggfunc='sum').fillna(0)
f.columns = ['무역점금액합계','본점금액합계','신촌점금액합계','천호점금액합계']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,21시할인금액합계,22시할인금액합계,무역점,본점,신촌점,천호점,무역점금액합계,본점금액합계,신촌점금액합계,천호점금액합계
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,5,6,0,0,460000.0,1282000.0,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,0.0,8,0,0,3,3161500.0,0.0,0.0,589350.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.0,0.0,4,0,0,26,462500.0,0.0,0.0,1838000.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,4,0,0,0,1045000.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.0,0.0,3,24,4,1,552000.0,3858459.0,391300.0,252000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0,4,0,0,0.0,632000.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,1,0,0,0,213000.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0,2,0,0,0.0,221000.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0,0,4,0,0.0,0.0,143029.0,0.0


In [37]:
f = pd.crosstab(df_all.custid, df_all.str_nm, values = df_all.dis_amt, aggfunc='sum').fillna(0)
f.columns = ['무역점할인금액합계','본점할인금액합계','신촌점할인금액합계','천호점할인금액합계']
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,신촌점,천호점,무역점금액합계,본점금액합계,신촌점금액합계,천호점금액합계,무역점할인금액합계,본점할인금액합계,신촌점할인금액합계,천호점할인금액합계
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,460000.0,1282000.0,0.0,0.0,46000.0,128200.0,0.0,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,3,3161500.0,0.0,0.0,589350.0,196150.0,0.0,0.0,58940.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0,26,462500.0,0.0,0.0,1838000.0,20440.0,0.0,0.0,71220.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,1045000.0,0.0,0.0,0.0,21800.0,0.0,0.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,4,1,552000.0,3858459.0,391300.0,252000.0,38600.0,282410.0,15050.0,25200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0.0,632000.0,0.0,0.0,0.0,31600.0,0.0,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,213000.0,0.0,0.0,0.0,10650.0,0.0,0.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0.0,221000.0,0.0,0.0,0.0,11050.0,0.0,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,4,0,0.0,0.0,143029.0,0.0,0.0,0.0,3500.0,0.0


In [38]:
f = pd.crosstab(df_all.custid, df_all.part_nm).fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,여성캐쥬얼,영라이브,영어덜트캐쥬얼,영캐릭터,영플라자,인터넷백화점,잡화,잡화파트,"케주얼,구두,아동",패션잡화
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,0,0,0,0,0,6,0,0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,0,1,0,0,0,1,0,0,0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0,7,3,0,0,0,0,0,0,0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,0,0,0,0,0,0,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0,0,0,0,0,0,0,4,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,0,0,0,0,2,2,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,0,0,0,0,0,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0,0,0,0,0,1,1,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,0,0,0,0,0,1


In [39]:
f = pd.crosstab(df_all.custid, df_all.team_nm).fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,영플라자,인터넷백화점_x,잡화,잡화파트,"케주얼,구두,아동",패션잡화,식품팀,의류패션팀,인터넷백화점_y,잡화가용팀
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,0,6,0,0,0,2,0,9
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,0,1,0,0,0,0,5,0,6
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0,0,0,0,0,0,3,13,0,14
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,0,0,0,0,0,3,0,1
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0,0,0,4,6,0,5,10,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,2,2,0,0,2,0,2
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,0,0,0,0,0,0,1
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0,1,1,0,0,1,0,1
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,0,1,2,0,0,2


In [40]:
f = pd.crosstab(df_all.custid, df_all.buyer_nm).fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,트래디셔널캐주얼,피혁A,피혁B,행사장(남성),행사장(아동스포츠),행사장(여성정장),행사장(여성캐주얼),행사장(여성캐쥬),행사장(잡화),화장품
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,1,0,0,0,0,0,0,0,0,8
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,3,1,0,0,0,0,0,0,0,1
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,1,1,0,0,0,0,0,0,0,1
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,0,0,0,0,0,0,0,1
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,4,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,0,0,0,0,0,0,1
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,0,0,0,0,0,0,1
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,1,0,0,0,0,0,0,0,1
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,0,0,0,0,0,1


분류별갯수

In [41]:
f = df_all.groupby('custid').agg({
    'str_nm': [('str_num', lambda x: x.nunique())],
    'brd_nm': [('brd_num', lambda x: x.nunique())],
    'corner_nm': [('corner_num', lambda x: x.nunique())],
    'pc_nm': [('pc_num', lambda x: x.nunique())],    
    'part_nm': [('part_num', lambda x: x.nunique())],
    'team_nm': [('team_num', lambda x: x.nunique())],
    'buyer_nm': [('buyer_num', lambda x: x.nunique())],
})

f.columns = f.columns.droplevel()
f=f.reset_index()

features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,행사장(여성캐쥬),행사장(잡화),화장품,str_num,brd_num,corner_num,pc_num,part_num,team_num,buyer_num
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,8,2,7,4,4,4,2,4
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,0,1,2,7,7,7,6,2,7
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0,0,1,2,21,18,14,8,3,14
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,1,1,4,4,3,3,2,3
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0,0,4,4,21,21,16,9,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,1,1,4,4,4,2,2,4
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,1,1,1,1,1,1,1,1
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,1,1,2,2,2,2,2,2
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,1,1,3,4,4,4,2,3


In [42]:
features['str_div'] = features['str_num']/df_all['str_nm'].nunique()
features['brd_div'] = features['brd_num']/df_all['brd_nm'].nunique()
features['corner_div'] = features['corner_num']/df_all['corner_nm'].nunique()
features['pc_div'] = features['pc_num']/df_all['pc_nm'].nunique()
features['part_div'] = features['part_num']/df_all['part_nm'].nunique()
features['team_div'] = features['team_num']/df_all['team_nm'].nunique()
features['buyer_div'] = features['buyer_num']/df_all['buyer_nm'].nunique()

features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,part_num,team_num,buyer_num,str_div,brd_div,corner_div,pc_div,part_div,team_div,buyer_div
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,4,2,4,0.50,0.003737,0.012987,0.051948,0.133333,0.50,0.114286
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,6,2,7,0.50,0.003737,0.022727,0.090909,0.200000,0.50,0.200000
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,8,3,14,0.50,0.011212,0.058442,0.181818,0.266667,0.75,0.400000
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,3,2,3,0.25,0.002136,0.012987,0.038961,0.100000,0.50,0.085714
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,9,3,11,1.00,0.011212,0.068182,0.207792,0.300000,0.75,0.314286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,2,2,4,0.25,0.002136,0.012987,0.051948,0.066667,0.50,0.114286
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,1,1,1,0.25,0.000534,0.003247,0.012987,0.033333,0.25,0.028571
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,2,2,2,0.25,0.001068,0.006494,0.025974,0.066667,0.50,0.057143
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,4,2,3,0.25,0.001602,0.012987,0.051948,0.133333,0.50,0.085714


In [43]:
f = df_all.groupby('custid').agg({
    'str_nm': [('str_max', lambda x: x.value_counts().index[0])],
    'brd_nm': [('brd_max', lambda x: x.value_counts().index[0])],
    'corner_nm': [('corner_max', lambda x: x.value_counts().index[0])],
    'pc_nm': [('pc_max', lambda x: x.value_counts().index[0])],   
    'part_nm': [('part_max', lambda x: x.value_counts().index[0])],
    'team_nm': [('team_max', lambda x: x.value_counts().index[0])],
    'buyer_nm': [('buyer_max', lambda x: x.value_counts().index[0])],
    'goodcd': [('goodcd_max', lambda x: x.value_counts().index[0])],
})

f.columns = f.columns.droplevel()
f=f.reset_index()

#features = features.merge(f, how='left'); features
df_all = df_all.merge(f, how='left'); df_all

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,...,inst_mon1,inst_fee1,str_max,brd_max,corner_max,pc_max,part_max,team_max,buyer_max,goodcd_max
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,...,3,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,...,1,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,...,3,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,...,3,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,...,3,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040034,49993,13,31,수,1750,신촌점,4405551020474,톰키드,아동,아동,...,1,0,신촌점,식품,차류,일반식품,아동문화,잡화가용팀,일반식품,2700000000000
1040035,49993,13,31,수,1833,신촌점,2139140008300,폴로화장품,향수,화장품,...,1,0,신촌점,식품,차류,일반식품,아동문화,잡화가용팀,일반식품,2700000000000
1040036,49994,16,14,토,1750,본점,4230120011274,스테파넬,영트랜드,영트렌디,...,1,0,본점,스테파넬,영트랜드,영트렌디,"케주얼,구두,아동",의류패션팀,영캐주얼,2116052008000
1040037,49994,16,14,토,1810,본점,4409271026010,써스데이아일앤드,진케주얼,진케주얼,...,1,0,본점,스테파넬,영트랜드,영트렌디,"케주얼,구두,아동",의류패션팀,영캐주얼,2116052008000


In [44]:
### 주구매지점 구매액

f = df_all.query("str_nm == str_max").groupby('custid')['tot_amt'].agg([
    ('str_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주브랜드 구매액

f = df_all.query("brd_nm == brd_max").groupby('custid')['tot_amt'].agg([
    ('brd_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주코너 구매액

f = df_all.query("corner_nm == corner_max").groupby('custid')['tot_amt'].agg([
    ('corner_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주상품군 구매액

f = df_all.query("pc_nm == pc_max").groupby('custid')['tot_amt'].agg([
    ('pc_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주파트 구매액

f = df_all.query("part_nm == part_max").groupby('custid')['tot_amt'].agg([
    ('part_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주관리팀 구매액

f = df_all.query("team_nm == team_max").groupby('custid')['tot_amt'].agg([
    ('team_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주바이어 구매액

f = df_all.query("buyer_nm == buyer_max").groupby('custid')['tot_amt'].agg([
    ('buyer_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left')

### 주상품 구매액

f = df_all.query("goodcd == goodcd_max").groupby('custid')['tot_amt'].agg([
    ('goodcd_tot', np.sum)
]).reset_index()

features = features.merge(f, how='left');features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,team_div,buyer_div,str_tot,brd_tot,corner_tot,pc_tot,part_tot,team_tot,buyer_tot,goodcd_tot
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.50,0.114286,1282000,533000,1056000,1056000,1282000,1511000,1056000,533000
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.50,0.200000,3161500,404500,404500,404500,2312000,2669350,1200000,1200000
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.75,0.400000,1838000,681000,816000,290200,1102000,1528500,1073000,25800
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.50,0.085714,1045000,60000,49000,425000,425000,985000,425000,60000
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.75,0.314286,3858459,343859,447000,391300,1923200,3089500,1148200,343859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.50,0.114286,632000,59000,52000,59000,157000,475000,59000,423000
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.25,0.028571,213000,213000,213000,213000,213000,213000,213000,213000
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.50,0.057143,221000,168000,53000,53000,53000,168000,53000,168000
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.50,0.085714,143029,53029,21000,21000,20000,90000,53029,53029


In [45]:
#구매등급
f = df_all.groupby('custid')['tot_amt'].agg([
    ('총구매액',np.sum),
    ('구매횟수',np.size)]).reset_index()

for i, row in f.iterrows() :
    if f.at[i,'총구매액'] >= 6000000 and f.at[i,'구매횟수'] >= 12:
        f.at[i,'구매등급'] =  'A'
    elif 4500000 <= f.at[i,'총구매액'] < 6000000 and f.at[i,'구매횟수'] >= 12:
        f.at[i,'구매등급'] =  'B'
    elif 3000000 <= f.at[i,'총구매액'] < 4500000 and f.at[i,'구매횟수'] >= 12:
         f.at[i,'구매등급'] = 'C'
    elif 1500000 <= f.at[i,'총구매액'] < 3000000 and f.at[i,'구매횟수'] >= 12:
        f.at[i,'구매등급'] = 'D'
    else :
        f.at[i,'구매등급'] = 'E'
f=f[['custid','구매등급']]
df_all = df_all.merge(f, how='left');df_all

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,...,inst_fee1,str_max,brd_max,corner_max,pc_max,part_max,team_max,buyer_max,goodcd_max,구매등급
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,...,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000,E
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,...,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000,E
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,...,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000,E
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,...,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000,E
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,...,0,본점,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,2116052008000,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040034,49993,13,31,수,1750,신촌점,4405551020474,톰키드,아동,아동,...,0,신촌점,식품,차류,일반식품,아동문화,잡화가용팀,일반식품,2700000000000,E
1040035,49993,13,31,수,1833,신촌점,2139140008300,폴로화장품,향수,화장품,...,0,신촌점,식품,차류,일반식품,아동문화,잡화가용팀,일반식품,2700000000000,E
1040036,49994,16,14,토,1750,본점,4230120011274,스테파넬,영트랜드,영트렌디,...,0,본점,스테파넬,영트랜드,영트렌디,"케주얼,구두,아동",의류패션팀,영캐주얼,2116052008000,E
1040037,49994,16,14,토,1810,본점,4409271026010,써스데이아일앤드,진케주얼,진케주얼,...,0,본점,스테파넬,영트랜드,영트렌디,"케주얼,구두,아동",의류패션팀,영캐주얼,2116052008000,E


In [46]:
f = pd.crosstab(df_all.custid, df_all.구매등급).fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,pc_tot,part_tot,team_tot,buyer_tot,goodcd_tot,A,B,C,D,E
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,1056000,1282000,1511000,1056000,533000,0,0,0,0,11
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,404500,2312000,2669350,1200000,1200000,0,0,0,0,11
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,290200,1102000,1528500,1073000,25800,0,0,0,30,0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,425000,425000,985000,425000,60000,0,0,0,0,4
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,391300,1923200,3089500,1148200,343859,0,32,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,59000,157000,475000,59000,423000,0,0,0,0,4
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,213000,213000,213000,213000,213000,0,0,0,0,1
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,53000,53000,168000,53000,168000,0,0,0,0,2
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,21000,20000,90000,53029,53029,0,0,0,0,4


In [47]:
f = pd.crosstab(df_all.custid, df_all.구매등급, values = df_all.tot_amt, aggfunc='sum').fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,A_x,B_x,C_x,D_x,E_x,A_y,B_y,C_y,D_y,E_y
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0,0,0,0,11,0.0,0.0,0.0,0.0,1742000.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,0,0,0,11,0.0,0.0,0.0,0.0,3750850.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0,0,0,30,0,0.0,0.0,0.0,2300500.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,0,0,4,0.0,0.0,0.0,0.0,1045000.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0,32,0,0,0,0.0,5053759.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,0,4,0.0,0.0,0.0,0.0,632000.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,0,0,1,0.0,0.0,0.0,0.0,213000.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,0,0,2,0.0,0.0,0.0,0.0,221000.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,4,0.0,0.0,0.0,0.0,143029.0


In [48]:
f = pd.crosstab(df_all.custid, df_all.구매등급, values = df_all.dis_amt, aggfunc='sum').fillna(0)
features = features.merge(f, on = 'custid'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,A_y,B_y,C_y,D_y,E_y,A,B,C,D,E
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,0.0,0.0,1742000.0,0.0,0.0,0.0,0.0,174200.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,0.0,0.0,0.0,3750850.0,0.0,0.0,0.0,0.0,255090.0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.0,0.0,0.0,2300500.0,0.0,0.0,0.0,0.0,91660.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,0.0,0.0,1045000.0,0.0,0.0,0.0,0.0,21800.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.0,5053759.0,0.0,0.0,0.0,0.0,361260.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0.0,632000.0,0.0,0.0,0.0,0.0,31600.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,0.0,213000.0,0.0,0.0,0.0,0.0,10650.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,0.0,221000.0,0.0,0.0,0.0,0.0,11050.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,143029.0,0.0,0.0,0.0,0.0,3500.0


In [49]:
x = df_all[df_all['import_flg'] == 1].groupby('custid').size() / df_all.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,B_y,C_y,D_y,E_y,A,B,C,D,E,수입상품_구매비율
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,0.0,1742000.0,0.0,0.0,0.0,0.0,174200.0,63.6
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,0.0,0.0,3750850.0,0.0,0.0,0.0,0.0,255090.0,9.1
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.0,0.0,2300500.0,0.0,0.0,0.0,0.0,91660.0,0.0,0.0
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,0.0,1045000.0,0.0,0.0,0.0,0.0,21800.0,25.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,5053759.0,0.0,0.0,0.0,0.0,361260.0,0.0,0.0,0.0,18.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,632000.0,0.0,0.0,0.0,0.0,31600.0,25.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,213000.0,0.0,0.0,0.0,0.0,10650.0,100.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,221000.0,0.0,0.0,0.0,0.0,11050.0,50.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,143029.0,0.0,0.0,0.0,0.0,3500.0,0.0


In [50]:
def f2(x):
    if x in ['월','화','수']:
        return('월화수_구매건수')
    elif x in ['목','금']:
        return('목금_구매건수')
    elif x in '토':
        return('토_구매건수')
    else :
        return('일_구매건수')    
    
df_all['요일2'] = df_all['sales_dayofweek'].apply(f2)
f = pd.pivot_table(df_all, index='custid', columns='요일2', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,A,B,C,D,E,수입상품_구매비율,목금_구매건수,월화수_구매건수,일_구매건수,토_구매건수
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,0.0,0.0,0.0,0.0,174200.0,63.6,2,2,5,2
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0.0,0.0,0.0,0.0,255090.0,9.1,2,5,4,0
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.0,0.0,0.0,91660.0,0.0,0.0,13,6,6,5
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0.0,0.0,0.0,0.0,21800.0,25.0,3,0,1,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.0,361260.0,0.0,0.0,0.0,18.8,23,4,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0.0,0.0,0.0,0.0,31600.0,25.0,0,4,0,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0.0,0.0,0.0,0.0,10650.0,100.0,0,1,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0.0,0.0,0.0,0.0,11050.0,50.0,1,0,1,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0.0,0.0,0.0,0.0,3500.0,0.0,0,4,0,0


In [51]:
def f1(x):
    if x in [14,15,16]:
        return('234월_구매건수')
    elif x in [5,6,7]:
        return('567월_구매건수')
    elif x in [8,9,10]:
        return('8910월_구매건수')
    else :
        return('11121월_구매건수')    
    
df_all['season2'] = df_all['sales_month'].apply(f1)
f = pd.pivot_table(df_all, index='custid', columns='season2', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,E,수입상품_구매비율,목금_구매건수,월화수_구매건수,일_구매건수,토_구매건수,11121월_구매건수,234월_구매건수,567월_구매건수,8910월_구매건수
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,174200.0,63.6,2,2,5,2,3,1,2,5
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,255090.0,9.1,2,5,4,0,4,0,0,7
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,0.0,0.0,13,6,6,5,6,7,10,7
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,21800.0,25.0,3,0,1,0,0,0,4,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,0.0,18.8,23,4,3,2,3,4,11,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,31600.0,25.0,0,4,0,0,4,0,0,0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,10650.0,100.0,0,1,0,0,0,1,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,11050.0,50.0,1,0,1,0,0,1,1,0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,3500.0,0.0,0,4,0,0,4,0,0,0


In [52]:
def f2(x):
    if 901 <= x < 1200 :
        return('12시 이전_구매건수')
    elif 1200 <= x < 1400 :
        return('12~2시_구매건수')
    elif 1400 <= x < 1600 :
        return('2~4시_구매건수')
    elif 1600 <= x < 1800 :
        return('4~6시_구매건수')
    else :
        return('6시이후_구매건수')  

df_all['timeslot2'] = df_all.sales_time.apply(f2)

f = pd.pivot_table(df_all, index='custid', columns='timeslot2', values='tot_amt',
                   aggfunc=np.size, fill_value=0).reset_index()
features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,토_구매건수,11121월_구매건수,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,2,3,1,2,5,2,0,0,0,9
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,4,0,0,7,1,3,1,2,4
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,5,6,7,10,7,5,3,3,6,13
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,0,4,0,0,0,4,0,0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,2,3,4,11,14,3,8,1,8,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,4,0,0,0,3,0,0,0,1
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,0,1,0,0,0,0,1,0,0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,0,1,1,0,0,0,0,1,1
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,4,0,0,0,0,0,0,3,1


In [53]:
f = df_all.groupby('custid')['sales_time'].agg([('평균구매시간', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,11121월_구매건수,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,3,1,2,5,2,0,0,0,9,1757.1
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,4,0,0,7,1,3,1,2,4,1537.1
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,6,7,10,7,5,3,3,6,13,1604.1
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,0,4,0,0,0,4,0,0,1434.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,3,4,11,14,3,8,1,8,12,1569.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,4,0,0,0,3,0,0,0,1,1372.5
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,0,1,0,0,0,0,1,0,0,1503.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,0,1,1,0,0,0,0,1,1,1796.5
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,4,0,0,0,0,0,0,3,1,1730.8


In [54]:
f = df_all.groupby('custid')['refund_tot'].agg([('환불액평균', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features = features.merge(f, how='left'); features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간,환불액평균
0,0,1742000,11,158363.636364,455000,174200,15836.363636,1567800,142527.272727,2.818182,...,1,2,5,2,0,0,0,9,1757.1,0.0
1,2,3750850,11,340986.363636,1416000,255090,23190.000000,3495760,317796.363636,3.454545,...,0,0,7,1,3,1,2,4,1537.1,168227.3
2,3,2300500,30,76683.333333,621000,91660,3055.333333,2208840,73628.000000,2.666667,...,7,10,7,5,3,3,6,13,1604.1,23186.7
3,4,1045000,4,261250.000000,560000,21800,5450.000000,1023200,255800.000000,4.500000,...,0,4,0,0,0,4,0,0,1434.0,0.0
4,5,5053759,32,157929.968750,682000,361260,11289.375000,4692499,146640.593750,1.875000,...,4,11,14,3,8,1,8,12,1569.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,31600,7900.000000,600400,150100.000000,2.250000,...,0,0,0,3,0,0,0,1,1372.5,0.0
35963,49990,213000,1,213000.000000,213000,10650,10650.000000,202350,202350.000000,6.000000,...,1,0,0,0,0,1,0,0,1503.0,0.0
35964,49992,221000,2,110500.000000,168000,11050,5525.000000,209950,104975.000000,1.000000,...,1,1,0,0,0,0,1,1,1796.5,0.0
35965,49993,143029,4,35757.250000,70000,3500,875.000000,139529,34882.250000,1.000000,...,0,0,0,0,0,0,3,1,1730.8,0.0


In [55]:
features = features.drop('custid', axis = 1)

### Data Cleansing & Feature Engineering

In [56]:
# 범주형 변수와 수치형 변수를 분리

cat_features = features.select_dtypes(include=['object']).columns.to_list()
num_features = features.select_dtypes(exclude='object').columns.to_list()

In [57]:
# 결측값 처리: 범주형이냐 수치형이냐에 따라 다르게 처리
from sklearn.impute import SimpleImputer 

if len(num_features) > 0: 
    features[num_features] = SimpleImputer(strategy='constant', fill_value=0).fit_transform(features[num_features])
if len(cat_features) > 0:  
    features[cat_features] = SimpleImputer(strategy="most_frequent").fit_transform(features[cat_features])

features

,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)","(inst_mon, 최대할부개월수)",...,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간,환불액평균
0,1742000.0,11.0,158363.636364,455000.0,174200.0,15836.363636,1567800.0,142527.272727,2.818182,3.0,...,1.0,2.0,5.0,2.0,0.0,0.0,0.0,9.0,1757.1,0.0
1,3750850.0,11.0,340986.363636,1416000.0,255090.0,23190.000000,3495760.0,317796.363636,3.454545,12.0,...,0.0,0.0,7.0,1.0,3.0,1.0,2.0,4.0,1537.1,168227.3
2,2300500.0,30.0,76683.333333,621000.0,91660.0,3055.333333,2208840.0,73628.000000,2.666667,5.0,...,7.0,10.0,7.0,5.0,3.0,3.0,6.0,13.0,1604.1,23186.7
3,1045000.0,4.0,261250.000000,560000.0,21800.0,5450.000000,1023200.0,255800.000000,4.500000,10.0,...,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,1434.0,0.0
4,5053759.0,32.0,157929.968750,682000.0,361260.0,11289.375000,4692499.0,146640.593750,1.875000,3.0,...,4.0,11.0,14.0,3.0,8.0,1.0,8.0,12.0,1569.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,632000.0,4.0,158000.000000,423000.0,31600.0,7900.000000,600400.0,150100.000000,2.250000,3.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1372.5,0.0
35963,213000.0,1.0,213000.000000,213000.0,10650.0,10650.000000,202350.0,202350.000000,6.000000,6.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1503.0,0.0
35964,221000.0,2.0,110500.000000,168000.0,11050.0,5525.000000,209950.0,104975.000000,1.000000,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1796.5,0.0
35965,143029.0,4.0,35757.250000,70000.0,3500.0,875.000000,139529.0,34882.250000,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1730.8,0.0


In [58]:
# 수치형 변수에 대해 이상치(outlier)를 처리

features[num_features] = features[num_features].apply(lambda x: x.clip(x.quantile(.05), x.quantile(.95)), axis=0)

In [59]:
# 수치형 변수를 정규분포에 가깝게 만들기 + 표준화
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer

mmscaler = MinMaxScaler()
rsscaler = RobustScaler()
#features[num_features] = mmscaler.fit_transform(features[num_features])
features[num_features] = np.log1p(features[num_features])
#features[num_features] = PowerTransformer(standardize=True).fit_transform(features[num_features])
#features[num_features] = rsscaler.fit_transform(features[num_features])

features

,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)","(inst_mon, 최대할부개월수)",...,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간,환불액평균
0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,1.386294,...,0.693147,1.098612,1.791759,1.098612,0.000000,0.000000,0.000000,2.302585,7.471989,0.000000
1,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,2.197225,...,0.000000,0.000000,2.079442,0.693147,1.386294,0.693147,1.098612,1.609438,7.338303,10.739952
2,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,1.791759,...,2.079442,2.397895,2.079442,1.791759,1.386294,1.386294,1.945910,2.639057,7.380941,10.051377
3,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,2.197225,...,0.000000,1.609438,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,7.268920,0.000000
4,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,1.386294,...,1.609438,2.484907,2.708050,1.386294,2.197225,0.693147,2.197225,2.564949,7.359086,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,13.356646,1.609438,11.970357,12.955130,10.360944,8.974745,13.305353,11.919064,1.178655,1.386294,...,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,0.693147,7.225118,0.000000
35963,12.269052,1.098612,12.269052,12.269052,9.273409,9.239061,12.217759,12.217759,1.386294,1.945910,...,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,7.315884,0.000000
35964,12.305923,1.098612,11.612780,12.031725,9.310276,8.617220,12.254629,11.561487,0.693147,0.693147,...,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,7.494152,0.000000
35965,11.870810,1.609438,10.484536,11.156265,8.160804,6.775366,11.846035,10.459762,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,7.456917,0.000000


In [60]:
# 범주형 변수에 One-Hot-Encoding 후 수치형 변수와 병합

if len(cat_features) > 0:
    features = pd.concat([features[num_features], pd.get_dummies(features[cat_features])], axis=1)
else:
    features = features[num_features]

features

,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)","(inst_mon, 최대할부개월수)",...,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간,환불액평균
0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,1.386294,...,0.693147,1.098612,1.791759,1.098612,0.000000,0.000000,0.000000,2.302585,7.471989,0.000000
1,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,2.197225,...,0.000000,0.000000,2.079442,0.693147,1.386294,0.693147,1.098612,1.609438,7.338303,10.739952
2,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,1.791759,...,2.079442,2.397895,2.079442,1.791759,1.386294,1.386294,1.945910,2.639057,7.380941,10.051377
3,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,2.197225,...,0.000000,1.609438,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,7.268920,0.000000
4,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,1.386294,...,1.609438,2.484907,2.708050,1.386294,2.197225,0.693147,2.197225,2.564949,7.359086,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,13.356646,1.609438,11.970357,12.955130,10.360944,8.974745,13.305353,11.919064,1.178655,1.386294,...,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,0.693147,7.225118,0.000000
35963,12.269052,1.098612,12.269052,12.269052,9.273409,9.239061,12.217759,12.217759,1.386294,1.945910,...,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,7.315884,0.000000
35964,12.305923,1.098612,11.612780,12.031725,9.310276,8.617220,12.254629,11.561487,0.693147,0.693147,...,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,7.494152,0.000000
35965,11.870810,1.609438,10.484536,11.156265,8.160804,6.775366,11.846035,10.459762,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,7.456917,0.000000


In [61]:
features.insert(0,'custid',df_all['custid'].unique())
features

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간,환불액평균
0,0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,...,0.693147,1.098612,1.791759,1.098612,0.000000,0.000000,0.000000,2.302585,7.471989,0.000000
1,2,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,...,0.000000,0.000000,2.079442,0.693147,1.386294,0.693147,1.098612,1.609438,7.338303,10.739952
2,3,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,...,2.079442,2.397895,2.079442,1.791759,1.386294,1.386294,1.945910,2.639057,7.380941,10.051377
3,4,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,...,0.000000,1.609438,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,7.268920,0.000000
4,5,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,...,1.609438,2.484907,2.708050,1.386294,2.197225,0.693147,2.197225,2.564949,7.359086,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,13.356646,1.609438,11.970357,12.955130,10.360944,8.974745,13.305353,11.919064,1.178655,...,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,0.693147,7.225118,0.000000
35963,49990,12.269052,1.098612,12.269052,12.269052,9.273409,9.239061,12.217759,12.217759,1.386294,...,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,7.315884,0.000000
35964,49992,12.305923,1.098612,11.612780,12.031725,9.310276,8.617220,12.254629,11.561487,0.693147,...,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,7.494152,0.000000
35965,49993,11.870810,1.609438,10.484536,11.156265,8.160804,6.775366,11.846035,10.459762,0.693147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,7.456917,0.000000


In [62]:
idx_train = pd.DataFrame({'custid': df_train['custid'].unique()})
X_train = pd.merge(idx_train, features, how='left', on='custid')

idx_test = pd.DataFrame({'custid': df_test['custid'].unique()})
X_test = pd.merge(idx_test, features, how='left', on='custid')

In [63]:
X_train

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,234월_구매건수,567월_구매건수,8910월_구매건수,12~2시_구매건수,12시 이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수,평균구매시간,환불액평균
0,0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,...,0.693147,1.098612,1.791759,1.098612,0.000000,0.000000,0.000000,2.302585,7.471989,0.000000
1,2,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,...,0.000000,0.000000,2.079442,0.693147,1.386294,0.693147,1.098612,1.609438,7.338303,10.739952
2,3,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,...,2.079442,2.397895,2.079442,1.791759,1.386294,1.386294,1.945910,2.639057,7.380941,10.051377
3,4,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,...,0.000000,1.609438,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,7.268920,0.000000
4,5,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,...,1.609438,2.484907,2.708050,1.386294,2.197225,0.693147,2.197225,2.564949,7.359086,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,16.129623,4.343805,12.278325,14.224968,12.819015,8.942196,16.093831,12.242107,1.035243,...,2.484907,3.178054,3.258097,2.890372,1.386294,2.833213,3.258097,2.772589,7.346204,9.784197
21583,29996,14.157184,2.995732,11.212758,12.384223,11.642207,8.697926,14.072863,11.128438,0.793231,...,2.772589,0.000000,0.000000,2.397895,0.000000,0.000000,1.791759,1.609438,7.320461,10.284120
21584,29997,14.996220,3.044522,12.000494,14.289880,10.604132,7.608871,14.983768,11.988042,0.916291,...,3.044522,0.000000,0.000000,1.609438,0.000000,2.484907,1.791759,0.000000,7.312687,0.000000
21585,29998,13.428231,2.639057,10.863299,11.695255,9.563178,6.999071,13.407047,10.842116,1.172720,...,0.693147,1.791759,1.098612,1.945910,0.000000,1.609438,1.098612,0.693147,7.285849,8.350383


### W2V(Corner_nm)

In [64]:
level = 'corner_nm' # 상품 분류 수준

# W2V 학습을 하기에는 데이터(즉 corpus)가 부족하여 
# 고객별로 구매한 상품 목록으로부터 20배 oversampling을 수행
sentences = []
df_all = pd.concat([df_train, df_test])
for id in df_all.custid.unique():
#    uw = df_all.query('custid == @id')[level].unique()
#    bs = np.array([])
#    for j in range(20):
#        bs = np.append(bs, np.random.choice(uw, len(uw), replace=False))
#    sentences.append(list(bs))
    sentences.append(list(df_all.query('custid == @id')[level].values))

In [65]:
max_features = 300 # 문자 벡터 차원 수
min_word_count = 1 # 최소 문자 수
num_workers = 4 # 병렬 처리 스레드 수
context = 3 # 문자열 창 크기
downsampling = 1e-3 # 문자 빈도수 Downsample

from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(sentences, 
                          workers=num_workers, 
                          vector_size=max_features, 
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)
# 필요없는 메모리 unload
model.init_sims(replace=True)

In [66]:
def age_vec():
    truth = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')

    sentences = []
    df_all = df_train
    for id in df_all.custid.unique():
        x = df_all.query('custid == @id')[level].unique()
        y = np.where(truth.query('custid == @id').age)
        for j in range(20):
            y = np.append(y, np.random.choice(x, len(x), replace=False))
        sentences.append(list(y))

In [67]:
# 고객별로 구매한 상품의 평균벡터를 feature로 사용한다.
features = []
for id in df_train.custid.unique():
    features.append(df_all.query('custid == @id')[level] \
                              .apply(lambda x: model.wv[x]).mean())
X_train_corner = np.array(features)

features = []
for id in df_test.custid.unique():
    features.append(df_all.query('custid == @id')[level] \
                              .apply(lambda x: model.wv[x]).mean())
X_test_corner = np.array(features)

In [68]:
X_train_corner = pd.DataFrame(X_train_corner)
X_test_corner = pd.DataFrame(X_test_corner)

In [69]:
idx_train = pd.DataFrame({'custid': df_train['custid'].unique()})
X_train_corner = pd.DataFrame(X_train_corner)
X_train_corner.insert(0,'custid',idx_train)

idx_test = pd.DataFrame({'custid': df_test['custid'].unique()})
X_test_corner = pd.DataFrame(X_test_corner)
X_test_corner.insert(0,'custid',idx_test)

In [70]:
X_train = pd.merge(X_train, X_train_corner, on='custid')
X_test = pd.merge(X_test, X_test_corner, on='custid')

In [71]:
X_train

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,290,291,292,293,294,295,296,297,298,299
0,0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,...,-0.002417,0.059097,0.018711,0.057068,0.014192,0.051211,0.037983,0.059910,0.013996,0.000925
1,2,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,...,-0.038651,-0.029063,0.019915,-0.035929,0.051000,0.009270,-0.011347,0.003275,-0.025213,0.007225
2,3,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,...,-0.003037,0.040846,0.010178,0.057005,0.043729,-0.003248,0.008260,0.006701,-0.039180,0.045709
3,4,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,...,-0.034724,0.037270,-0.029957,0.053158,-0.027944,0.073096,0.010937,0.034829,0.043488,0.047925
4,5,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,...,0.028290,0.021322,0.011792,-0.018147,0.031468,0.023915,0.005695,-0.005596,0.008757,-0.000172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,16.129623,4.343805,12.278325,14.224968,12.819015,8.942196,16.093831,12.242107,1.035243,...,-0.018821,0.031391,0.020302,-0.009677,0.016187,0.040115,0.031400,0.026931,0.012116,0.006041
21583,29996,14.157184,2.995732,11.212758,12.384223,11.642207,8.697926,14.072863,11.128438,0.793231,...,-0.021279,-0.027551,-0.019851,0.020414,-0.014950,0.031239,0.001450,0.013004,0.034320,-0.001755
21584,29997,14.996220,3.044522,12.000494,14.289880,10.604132,7.608871,14.983768,11.988042,0.916291,...,0.027181,-0.007617,0.024105,-0.028796,0.049956,0.019912,0.027706,-0.019908,-0.015445,-0.014149
21585,29998,13.428231,2.639057,10.863299,11.695255,9.563178,6.999071,13.407047,10.842116,1.172720,...,-0.012761,-0.005869,-0.012231,-0.015481,0.038144,0.016625,0.002647,-0.008706,0.025253,-0.028587


### W2V (brd_nm)

In [72]:
### Make corpus
level = 'brd_nm' # 상품 분류 수준

# W2V 학습을 하기에는 데이터(즉 corpus)가 부족하여 
# 고객별로 구매한 상품 목록으로부터 20배 oversampling을 수행
sentences = []
df_all = pd.concat([df_train, df_test])
for id in df_all.custid.unique():
#    uw = df_all.query('custid == @id')[level].unique()
#    bs = np.array([])
#    for j in range(20):
#        bs = np.append(bs, np.random.choice(uw, len(uw), replace=False))
#    sentences.append(list(bs))
    sentences.append(list(df_all.query('custid == @id')[level].values))

### Training the Word2Vec model
max_features = 300 # 문자 벡터 차원 수
min_word_count = 1 # 최소 문자 수
num_workers = 4 # 병렬 처리 스레드 수
context = 3 # 문자열 창 크기
downsampling = 1e-3 # 문자 빈도수 Downsample

from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(sentences, 
                          workers=num_workers, 
                          vector_size=max_features, 
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)
# 필요없는 메모리 unload
model.init_sims(replace=True)

In [73]:
def age_vec():
    truth = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')

    sentences = []
    df_all = df_train
    for id in df_all.custid.unique():
        x = df_all.query('custid == @id')[level].unique()
        y = np.where(truth.query('custid == @id').age)
        for j in range(20):
            y = np.append(y, np.random.choice(x, len(x), replace=False))
        sentences.append(list(y))

In [74]:
# 고객별로 구매한 상품의 평균벡터를 feature로 사용한다.
features = []
for id in df_train.custid.unique():
    features.append(df_all.query('custid == @id')[level] \
                              .apply(lambda x: model.wv[x]).mean())
X_train_brd = np.array(features)

features = []
for id in df_test.custid.unique():
    features.append(df_all.query('custid == @id')[level] \
                              .apply(lambda x: model.wv[x]).mean())
X_test_brd = np.array(features)

In [75]:
X_train_brd = pd.DataFrame(X_train_brd)
X_test_brd = pd.DataFrame(X_test_brd)

In [76]:
idx_train = pd.DataFrame({'custid': df_train['custid'].unique()})
X_train_brd = pd.DataFrame(X_train_brd)
X_train_brd.insert(0,'custid',idx_train)

idx_test = pd.DataFrame({'custid': df_test['custid'].unique()})
X_test_brd = pd.DataFrame(X_test_brd)
X_test_brd.insert(0,'custid',idx_test)

In [77]:
X_train = pd.merge(X_train, X_train_brd, on='custid')
X_test = pd.merge(X_test, X_test_brd, on='custid')

In [78]:
X_train

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,...,0.019055,0.035606,0.026098,0.041948,0.078150,0.055885,0.016683,-0.019240,-0.002032,-0.062671
1,2,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,...,0.043073,0.028162,0.004513,0.041677,0.072565,0.063608,-0.030890,-0.030114,0.027990,-0.055665
2,3,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,...,0.024491,0.054740,-0.007923,0.018778,0.047326,0.057957,-0.039040,-0.014993,-0.014448,-0.043463
3,4,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,...,-0.024256,0.014604,-0.022314,-0.006741,-0.005523,0.044205,-0.028516,0.010196,-0.007311,-0.047189
4,5,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,...,-0.018713,0.036353,0.018997,0.004109,0.016315,0.034833,0.003166,-0.011647,0.018216,-0.047603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,16.129623,4.343805,12.278325,14.224968,12.819015,8.942196,16.093831,12.242107,1.035243,...,-0.036094,0.054512,0.008776,0.054717,0.033665,0.041246,0.020268,-0.014632,0.003757,-0.029378
21583,29996,14.157184,2.995732,11.212758,12.384223,11.642207,8.697926,14.072863,11.128438,0.793231,...,0.061289,0.017036,-0.006296,0.034454,-0.000701,0.020697,-0.060943,-0.018907,0.018978,-0.064430
21584,29997,14.996220,3.044522,12.000494,14.289880,10.604132,7.608871,14.983768,11.988042,0.916291,...,-0.009039,0.033519,0.017380,-0.010956,0.036151,0.032377,-0.021939,-0.034716,0.065368,-0.026993
21585,29998,13.428231,2.639057,10.863299,11.695255,9.563178,6.999071,13.407047,10.842116,1.172720,...,0.002910,0.049178,0.000481,0.059229,0.069509,0.066698,-0.017682,-0.025183,-0.007997,-0.087287


In [79]:
X_test

,custid,"(tot_amt, 총구매액)","(tot_amt, 구매건수)","(tot_amt, 평균구매가격)","(tot_amt, 최대구매액)","(dis_amt, dis_sum)","(dis_amt, dis_mean)","(net_amt, net_sum)","(net_amt, net_mean)","(inst_mon, 평균할부개월수)",...,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,30001,15.240622,3.332205,11.944791,13.607257,11.679041,8.383425,15.211817,11.915987,1.008228,...,-0.026819,0.052060,0.012303,0.021599,0.014717,0.016175,0.001004,-0.021855,0.042156,-0.048157
1,30002,15.903696,4.543295,11.278736,12.923915,12.660490,8.035838,15.863875,11.238915,1.118030,...,0.006817,0.040790,-0.009103,0.027915,0.041831,0.027648,-0.006224,-0.008788,-0.010001,-0.038235
2,30003,14.147416,3.871201,10.316067,11.835016,9.922849,6.111467,14.132677,10.287882,0.850333,...,0.022436,0.035253,0.005384,0.009714,0.050788,0.009884,-0.030906,-0.018467,0.020992,-0.036704
3,30005,11.790081,1.098612,10.950824,11.141876,8.160804,7.955425,11.746819,10.899532,1.386294,...,-0.001921,0.073069,0.035383,0.011010,0.011024,0.001447,0.028944,0.019644,-0.051744,-0.085140
4,30007,12.596427,1.945910,10.804685,11.141876,9.220390,7.429125,12.561650,10.769909,0.847298,...,0.021863,0.052785,-0.020461,0.060755,0.107149,0.047394,0.006994,-0.001709,-0.019526,-0.058733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,49988,13.356646,1.609438,11.970357,12.955130,10.360944,8.974745,13.305353,11.919064,1.178655,...,0.026747,0.061195,0.021727,-0.039691,0.049992,0.016169,0.024157,0.024926,0.061604,0.012418
14376,49990,12.269052,1.098612,12.269052,12.269052,9.273409,9.239061,12.217759,12.217759,1.386294,...,0.127433,0.024156,0.065003,0.015225,0.099149,0.105248,-0.022254,-0.047367,-0.037097,-0.085470
14377,49992,12.305923,1.098612,11.612780,12.031725,9.310276,8.617220,12.254629,11.561487,0.693147,...,-0.001227,0.037518,-0.103792,0.012161,-0.006290,0.045153,-0.006920,-0.001745,-0.057985,-0.050592
14378,49993,11.870810,1.609438,10.484536,11.156265,8.160804,6.775366,11.846035,10.459762,0.693147,...,0.000161,0.001769,0.030943,-0.036691,0.052809,0.018524,-0.043302,-0.036900,0.075794,0.021218


In [80]:
X_train.to_csv('jin_features.csv' , index = False )
X_test.to_csv('jin_features_te.csv', index = False )